In [12]:
import dolfin as dolf
import numpy as np

In [13]:
np.set_printoptions(precision=3)

In [14]:
# Example for the stiffness matrix C
# f_1 for real numbers and f_2 for complex numbers
# We don't apply boundary conditions

In [15]:
def f_1():
    
    mesh = dolf.UnitIntervalMesh(2)
    
    V = dolf.FunctionSpace(mesh, 'CG', 1)

    u = dolf.TrialFunction(V)
    v = dolf.TestFunction(V)

    dx = dolf.Measure('dx', domain=mesh)

    bcs = []
    
    c_ = u * v * dx
    dummy = v * dx

    C, b = dolf.assemble_system(c_, dummy, bcs)
    C = dolf.as_backend_type(C).mat()
    
    return C

In [16]:
def f_2():

    mesh = dolf.UnitIntervalMesh(2)

    CG = dolf.FiniteElement('CG', mesh.ufl_cell(), 1)
    W = dolf.FunctionSpace(mesh, dolf.MixedElement([CG, CG]))

    (u_1, u_2) = dolf.TrialFunction(W)
    (v_1, v_2) = dolf.TestFunction(W)

    dx = dolf.Measure('dx', domain=mesh)

    bcs = []

    c_11 = v_1 * u_1 * dx
    c_22 = v_2 * u_2 * dx
    c_ = c_11 + c_22
    dummy = (v_1 + v_2) * dx

    C, b = dolf.assemble_system(c_, dummy, bcs)
    C = dolf.as_backend_type(C).mat()
    
    return C

In [17]:
C = f_1()  # 3x3 matrix
C.getValues(range(3), range(3))

array([[0.167, 0.083, 0.   ],
       [0.083, 0.333, 0.083],
       [0.   , 0.083, 0.167]])

In [18]:
C = f_2()  # 6x6 matrix
C.getValues(range(6), range(6))

array([[0.167, 0.   , 0.083, 0.   , 0.   , 0.   ],
       [0.   , 0.167, 0.   , 0.083, 0.   , 0.   ],
       [0.083, 0.   , 0.333, 0.   , 0.083, 0.   ],
       [0.   , 0.083, 0.   , 0.333, 0.   , 0.083],
       [0.   , 0.   , 0.083, 0.   , 0.167, 0.   ],
       [0.   , 0.   , 0.   , 0.083, 0.   , 0.167]])

In [19]:
# Can you see the block structure? In case you can't see it, I will remap C into K

In [20]:
C = C.getValues(range(6), range(6))
K = np.zeros_like(C)
for i in range(3):
    for j in range(3):
        C_ij = C[2*i:2*i+2,2*j:2*j+2]
        K[i, j] = C_ij[0, 0]
        K[i, j+3] = C_ij[0, 1]
        K[i+3, j] = C_ij[1, 0]
        K[i+3, j+3] = C_ij[1, 1]

In [21]:
K

array([[0.167, 0.083, 0.   , 0.   , 0.   , 0.   ],
       [0.083, 0.333, 0.083, 0.   , 0.   , 0.   ],
       [0.   , 0.083, 0.167, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.167, 0.083, 0.   ],
       [0.   , 0.   , 0.   , 0.083, 0.333, 0.083],
       [0.   , 0.   , 0.   , 0.   , 0.083, 0.167]])

In [22]:
# Do 'c_11 = v_1 * u_1 * dx' and 'c_22 = v_2 * u_2 * dx' make more sense now?